# Segmentation and Visualisation of Rodent Mitochondria

In [0]:
from IPython.display import HTML

HTML('''<script>
code_show=false; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### Objective

The purpose of this notebook is to go through some automated analysis processes of morphology one might want to measure during a typical study of mitochondria

This notebook draws upon literature from the following sources for analysis: 

    Vincent et al. Cell Reports 26, 996–1009, January 22, 2019
    Lucchi et al. IEEE Transactions on Medical Imaging. 2015.
    Luchhi et al. CVPR, Portland, Oregon, USA, June 23}-28, 2013
    Belevich et al. PLOS Biology | DOI:10.1371/journal.pbio.1002340

The dataset can be downloaded from: https://cvlab.epfl.ch/research/page-90578-en-html/research-medical-em-mitochondria-index-php/

### Some context 
Morphological changes caused by genetic and biochemical mitochondrial funcion defects are a major cause of human disease. Mitochondria are subject to continious morphological changes over time. Therefore quantifying these parameters in an automated manner at high throughput is key in understanding mitochondrial diseases. 

One of the approaches to measure these morphological changes at high resolution is by using serial block face scanning electron microscopy. More information about this technique can be found in the link: https://en.wikipedia.org/wiki/Serial_block-face_scanning_electron_microscopy. 

In short, this technique slices the samples into sections using an ultramicrotome and preiodically records a scanning electron microscope image. The images can be stacked together to obtain a 3D reconstruction of the microtomed region. 

### Downloading the dataset from google drive

In [0]:
#taken from this StackOverflow answer: https://stackoverflow.com/a/39225039
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [0]:
import os
# link to download the pretrained CNN model
file_id = '1PUkTh6atShukHKYDqOrd-TYEFRS73hff'
destination = os.path.join(os.getcwd(),'model.h5')
download_file_from_google_drive(file_id, destination)
print("downloaded the model")


# link to download the volumetric data https://drive.google.com/file/d/1JM-AUC5pikeaYMrALmGKMjpY5fcux4tO/view?usp=sharing
file_id = '1JM-AUC5pikeaYMrALmGKMjpY5fcux4tO'
destination = os.path.join(os.getcwd(),'volumedata.tif')
download_file_from_google_drive(file_id, destination)
print("downloaded the fib data")

# link to download the training dataset https://drive.google.com/file/d/1u6Ut_BJ4VBeJw8zn13NDjPErGDRbAZsQ/view?usp=sharing
file_id = '1u6Ut_BJ4VBeJw8zn13NDjPErGDRbAZsQ'
destination = os.path.join(os.getcwd(),'training.tif')
download_file_from_google_drive(file_id, destination)
print("downloaded the training data")


#link to download the test dataset https://drive.google.com/file/d/1RiMNJxa8TK1MTHdL-xY-sx_w_x3WQxct/view?usp=sharing
file_id = '1RiMNJxa8TK1MTHdL-xY-sx_w_x3WQxct'
destination = os.path.join(os.getcwd(),'training_groundtruth.tif')
download_file_from_google_drive(file_id, destination)
print("downloaded the ground truth")

#link to download the test dataset https://drive.google.com/file/d/1hX6-kNG_6N_p7330slD-jNpM2i59Jc_G/view?usp=sharing
file_id = '1hX6-kNG_6N_p7330slD-jNpM2i59Jc_G'
destination = os.path.join(os.getcwd(),'testing.tif')
download_file_from_google_drive(file_id, destination)
print("downloaded the testing data")

#link to download the test dataset https://drive.google.com/file/d/1XFovvC_d7NK7Xi0xhndp7ng40GlJa8Z_/view?usp=sharing
file_id = '1XFovvC_d7NK7Xi0xhndp7ng40GlJa8Z_'
destination = os.path.join(os.getcwd(),'testing_groundtruth.tif')
download_file_from_google_drive(file_id, destination)
print("downloaded the testing ground truth")

![alt text](https://assets.datacamp.com/production/repositories/3981/datasets/b22fa4767f979b26b193faa2cf166adde3d6cf54/Screen%20Shot%202019-01-19%20at%202.53.18%20PM.png)

### The dataset of interest

The image stack represents a 5x5x5µm section taken from the CA1 hippocampus region of the brain, corresponding to a 1065x2048x1536 volume with a voxel resolution of ~5nm. 

We can visualise this dataset along the x, y and z directions as a function of depth within the sample as follows:

In [0]:
from skimage.io import imread
dataset3D = imread('volumedata.tif') # read the tiff image stack
print(dataset3D.shape) # check the image stack shape to see whether it loaded properly

Visualisation of the 3D dataset from the  publication: https://www.sciencedirect.com/science/article/pii/S0968432814000250

![alt text](https://ars.els-cdn.com/content/image/1-s2.0-S0968432814000250-gr1.jpg)



In [0]:
import numpy as np
import matplotlib.pyplot as plt

pixel_width = 5 # stated in the dataset record 
pixel_units = 'nm'

nseg = 7 # splitting the dataset into 7 segments for viewing
segsx = np.arange(0,dataset3D.shape[0],int(dataset3D.shape[0]/nseg))
segsy = np.arange(0,dataset3D.shape[1],int(dataset3D.shape[1]/nseg))
segsz = np.arange(0,dataset3D.shape[2],int(dataset3D.shape[2]/nseg))
segs = np.ravel(np.asarray([segsx,segsy,segsz]))

# plotting all 7 segments along x y and z direction
plt.figure(figsize=(15,5))
for i in range(1,22):
    plt.subplot(3,7,i)
    if i < 8:
        plt.title(str(segs[i]*pixel_width)+pixel_units+" along x")
        plt.imshow(dataset3D[segs[i],:,:],cmap='gray')
    if i > 7 and i < 15:
        plt.title(str(segs[i]*pixel_width)+pixel_units+" along y")
        plt.imshow(dataset3D[:,segs[i],:],cmap='gray')
    if i > 14:
        plt.title(str(segs[i]*pixel_width)+pixel_units+" along z")
        plt.imshow(dataset3D[:,:,segs[i]],cmap='gray')        
    plt.axis('off')
plt.tight_layout()
plt.savefig('tomographs.png',dpi=100)
plt.show()

### Segmentation of the dataset.
To obtain quantitative information for the sample, the mitochondria need to be segmented from the images. There are numerous freeware and commercial packes to perform segmentation such as ImageJ, Amira, BioImageXD etc... 

However most packages have either sub-optimal segmentation routines, or require files in proprietary form or have laborious and time consuming workflows. Because of this, recent efforts have been geared towards building computer vision models to identify and segment images using deep learning.

For this very purpose, challenges such as ImageNet have emerged and demonstrate the capabilities of of the aforementioned method in comparison with other methods based on support vector machines, Markov models etc. 

For further reading please refer to the publication here: https://arxiv.org/ftp/arxiv/papers/1605/1605.09612.pdf

For this dataset a simple U-Net architecture (called this because the network resembls a U shape) is used. 

In [0]:
from tensorflow import keras

# defining the different convolutional blocks
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

#defining the unet model
def UNet(image_size):
    f = [16, 32, 64, 128, 256]

    inputs = keras.layers.Input((image_size, image_size, 1))
    s = keras.layers.Lambda(lambda x: x / 255) (inputs)

    p0 = s
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8

    bn = bottleneck(p4, f[4])

    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128

    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)

    return model

![alt text](http://meetshah1995.github.io/images/blog/ss/grfnet.png)

Example of a GFR-Net architecture https://meetshah1995.github.io/semantic-segmentation/deep-learning/pytorch/visdom/2017/06/01/semantic-segmentation-over-the-years.html

In [0]:
model = UNet(256) #create the model

In [0]:
model.summary()

In [0]:
model.load_weights('model.h5') # to speed up training lets use weights from a pre-trained model

The model can be visualised using Kera's plot_model function as shown below

In [0]:
from tensorflow.keras.utils import plot_model
plot_model(model,to_file='model.png') # plot the model 
plt.figure(figsize=(5,1),dpi=700)
plt.imshow(np.rot90(imread('model.png')))
plt.axis('off')
plt.tight_layout()
plt.show()

To train this model to 'learn' where the mitochondria are within the images, a few requirements must be met. 
1. A set of training data with annotated labels is required
2. The training dataset should be generalised in order for the model to be robust when making predictions on 'live' or validation data.

There are also other requirements such as the model should not overfit ot how the model should deal with sub-classes belonging to multiple datasets or even the ampunt of data needed to train the model. A comprehensive list of these can be found at: https://ti.arc.nasa.gov/m/pub-archive/archive/0473.pdf

For this dataset a set of training and validation data was generously provided by https://cvlab.epfl.ch/research/page-90578-en-html/research-medical-em-mitochondria-index-php/.

Some of the visualisations of the training and testing data are provided below:

In [0]:
from skimage.color import label2rgb

X_train = imread('training.tif')
y_train = imread('training_groundtruth.tif')
X_test  = imread('testing.tif')
y_test  = imread('testing_groundtruth.tif')

np.random.seed(47) # fixing the seed for reproducibility
idx = np.random.randint(0,X_train.shape[0]-1,10)

plt.figure(figsize=(10,5))
plt.suptitle("Training images selected at random",fontsize=32)
for i in range(0,10):
    plt.subplot(2,5,i+1)
    plt.imshow(label2rgb(y_train[idx[i]],X_train[idx[i]]),
               alpha=0.7)
    plt.axis('off')
plt.tight_layout()
plt.savefig('test_data.png',dpi=100)
plt.show()

plt.figure(figsize=(10,5))
plt.suptitle("Testing images selected at random",fontsize=32)
for i in range(0,10):
    plt.subplot(2,5,i+1)
    plt.imshow(label2rgb(y_test[idx[i]],X_test[idx[i]]),
               alpha=0.7)
    plt.axis('off')
plt.tight_layout()
plt.savefig('training_data.png',dpi=100)
plt.show()

To train the UNet model with this data, the images and labels need to be resized into 1x256x256 (this is the input the model was designed to accept). 

After the data has been resized, the data can be used as an input for the UNet model. However in some cases it is useful to 'augment' the data to improve the models ability to generalise to problems. 

More details in data augmentation can be found at: https://openreview.net/pdf?id=rkBBChjiG

For this dataset some simple augmentations were applied at random. These were: Gaussian Blurring, Gaussian Noise, Contrast Normalisation, Addition and Subtraction of intensities. 

In [0]:
from skimage.transform import resize
from tqdm import tqdm_notebook as tqdm
X_train_resized = np.zeros((X_train.shape[0],256,256,1),dtype='uint8')
y_train_resized = np.zeros((y_train.shape[0],256,256,1),dtype='bool')
X_test_resized  = np.zeros((X_test.shape[0],256,256,1),dtype='uint8')
y_test_resized  = np.zeros((y_test.shape[0],256,256,1),dtype='bool')

#resizing the dataset
for i in tqdm(range(0,X_train.shape[0])):
    X_train_resized[i] = resize(X_train[i],(256,256,1),mode='constant',preserve_range=True)
    y_train_resized[i] = resize(y_train[i],(256,256,1),mode='constant',preserve_range=True)
    X_test_resized[i] = resize(X_test[i],(256,256,1),mode='constant',preserve_range=True)
    y_test_resized[i] = resize(y_test[i],(256,256,1),mode='constant',preserve_range=True)

In [0]:
# applying augmentations to the dataset 
import imgaug as ia
from imgaug import augmenters as iaa
from sklearn.utils import shuffle

def augment_dataset(X,y):
    sometimes = lambda aug: iaa.Sometimes(1, aug) # e.g. Sometimes(0.5, ...) applies the given augmenter in 50% of all cases
    seq = iaa.Sequential([
        sometimes(iaa.GaussianBlur(sigma=(0, 0.3))), # blur images with a sigma of 0 to 3.0
        sometimes(iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5)),
        sometimes(iaa.AdditivePoissonNoise()),
        sometimes(iaa.ContrastNormalization((0.5, 2.0))),
        sometimes(iaa.Add((-10, 10))),
    ],random_order=True)
    return shuffle(np.concatenate((X,seq.augment_images(X))),
           np.concatenate((y,y)),random_state=42)
    
X_train_augmented, y_train_augmented = augment_dataset(X_train_resized,y_train_resized) 

In [0]:
# visualising the augmentations
idx = np.random.randint(0,X_train_augmented.shape[0]-1,10)

plt.figure(figsize=(12.5,7))
plt.suptitle("Augmented - Training images selected at random",fontsize=32)
for i in range(0,10):
    plt.subplot(2,5,i+1)
    plt.imshow(label2rgb(y_train_augmented[idx[i],:,:,0],X_train_augmented[idx[i],:,:,0]),
               alpha=0.7)
    plt.axis('off')
plt.tight_layout()
plt.savefig('test_data.png',dpi=100)
plt.show()

When evaluating a standard machine learning model, we usually classify our predictions into four categories: true positives, false positives, true negatives, and false negatives. However, for the dense prediction task of image segmentation, it's not immediately clear what counts as a "true positive" and, more generally, how we can evaluate our predictions? we define an intersect over union metric. https://www.jeremyjordan.me/evaluating-image-segmentation-models/

This is basically an overlap test to benchmark the models performance. Finally, with all the ingredients in place the model can be trained on the augmented dataset. 

In [0]:
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-mitochondria.h5', verbose=1, save_best_only=True)

In [0]:
#create new tensorboard logs. call "tensorboard --logdir logs/fit" to view logs
import datetime
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(X_train_augmented,y_train_augmented,
          validation_split=0.1,
          batch_size=8,
          epochs=1,
          callbacks=[earlystopper, checkpointer, tensorboard_callback]);

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
%tensorboard --logdir logs/fit

In [0]:
# plotting the accuracy and loss of the model over time
# import plotly objects and generate the graphs in offline mode
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

enable_plotly_in_cell()

epochs = np.linspace(0,len(model.history.history['loss']),len(model.history.history['loss'])+1)
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']
mean_iou = model.history.history['mean_iou']
val_mean_iou = model.history.history['val_mean_iou']

trace1 = go.Scatter(x=epochs,y=loss,name="loss")
trace2 = go.Scatter(x=epochs,y=val_loss,name="validation loss")
trace3 = go.Scatter(x=epochs,y=mean_iou,name="mean iou")
trace4 = go.Scatter(x=epochs,y=val_mean_iou,name="validation mean iou")

fig = tools.make_subplots(rows=1, cols=2)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)
fig.append_trace(trace3, 1, 2)
fig.append_trace(trace4, 1, 2)
fig['layout'].update(height=400, 
                     width=900, 
                     title='Comparison of training and validation progress with increasing epochs')
fig['layout']['xaxis1'].update(title='epoch')
fig['layout']['xaxis2'].update(title='epoch')
fig['layout']['yaxis1'].update(title='loss')
fig['layout']['yaxis2'].update(title='mean iou')
iplot(fig)

The graphs above show that the mean intersection metric caps out at ~>0.8 in the validaiton test.To improve the metric, there are several ways to update the model from here to increase accuracy. But in this case we want to check this simple model's performance on the testing data by visualising the hand segmented labels against the trained model's predictions. The output from the model can be interpreted as a probability map of how confident the model is at assigning the pixels belonging to the mitochondria. For this a probability >0.5 was chosen as the threshold for classifying mitochondria pixels. 

In [0]:
y_test_pred = model.predict(X_test_resized,verbose=1)

In [0]:
idx = np.random.randint(0,y_test_pred.shape[0]-1,5)

plt.figure(figsize=(20,5))
plt.suptitle("Predictions from the model",fontsize=32)
for i in range(0,5):
    plt.subplot(1,5,i+1)
    plt.imshow(label2rgb(y_test_pred[idx[i],:,:,0]>0.5,X_test_resized[idx[i],:,:,0]),alpha=0.7)
    plt.axis('off')
plt.tight_layout()
plt.show()

plt.figure(figsize=(20,5))
plt.suptitle("Manually segmented labels",fontsize=32)
for i in range(0,5):
    plt.subplot(1,5,i+1)
    plt.imshow(label2rgb(y_test_resized[idx[i],:,:,0]>0.5,X_test_resized[idx[i],:,:,0]),alpha=0.7)
    plt.axis('off')
plt.tight_layout()
plt.show()

A comparison of the model against the manually segmented data shows that the model still needs refining in classifying the mitochondria in some regions of the images. Some popular methods to improve the segmentation are to introduce more relevant augmentations, addition of data with false positives, adding dropout layers in the model for generalisation, using a more sophisticated model etc. 

As the results look sensible for this specific case, the model can be used to segment the three dimensional dataset.

### Three dimensional segmentation of the ultra microtome section

In [0]:
#prep 3D data for the model to make predictions
def prepare_3Ddata(array3D,imw,imh):
    if array3D.dtype=='bool':
        data = np.zeros((array3D.shape[2],imw,imh,1),dtype='bool')
    else:
        data = np.zeros((array3D.shape[2],imw,imh,1),dtype='uint8')
    for i in tqdm(range(0,data.shape[0])):
        data[i] = resize(array3D[:,:,i],(imw,imh,1),mode='constant',preserve_range=True)
    return data

In [0]:
dataset3D_resized = prepare_3Ddata(dataset3D,256,256)

In [0]:
dataset3D_pred = model.predict(dataset3D_resized,verbose=1)

In [0]:
dataset3D_pred_resized = np.zeros((dataset3D.shape[0],
                                   dataset3D.shape[1],
                                   dataset3D.shape[2]),dtype='bool')

for i in tqdm(range(0,dataset3D_pred_resized.shape[0])):
    dataset3D_pred_resized[:,:,i] = resize(dataset3D_pred[i,:,:,0]>0.5,
                                          (dataset3D.shape[0],dataset3D.shape[1]),
                                           preserve_range=True)

In [0]:
nseg = 7 # splitting the dataset into 7 segments for viewing
segsx = np.arange(0,dataset3D.shape[0],int(dataset3D.shape[0]/nseg))
segsy = np.arange(0,dataset3D.shape[1],int(dataset3D.shape[1]/nseg))
segsz = np.arange(0,dataset3D.shape[2],int(dataset3D.shape[2]/nseg))
segs = np.ravel(np.asarray([segsx,segsy,segsz]))

# plotting all 7 segments along x y and z direction
plt.figure(figsize=(20,5))
for i in range(1,22):
    plt.subplot(3,7,i)
    if i < 8:
        plt.title(str(segs[i]*pixel_width)+pixel_units+" along x")
        plt.imshow(label2rgb(dataset3D_pred_resized[segs[i],:,:],dataset3D[segs[i],:,:]),
                   alpha=0.7,cmap='gray')
    if i > 7 and i < 15:
        plt.title(str(segs[i]*pixel_width)+pixel_units+" along y")
        plt.imshow(label2rgb(dataset3D_pred_resized[:,segs[i],:],dataset3D[:,segs[i],:]),
                   alpha=0.7,cmap='gray')
    if i > 14:
        plt.title(str(segs[i]*pixel_width)+pixel_units+" along z")
        plt.imshow(label2rgb(dataset3D_pred_resized[:,:,segs[i]],dataset3D[:,:,segs[i]]),
                   alpha=0.7,cmap='gray')        
    plt.axis('off')
plt.tight_layout()
plt.savefig('tomographs_labelled.png',dpi=100)
plt.show()

In [0]:
#clearing redundant variables and cleaning up variable space
import gc
gc.collect()
del dataset3D_resized, y_test_resized, X_test_resized, 
del X_train, y_train, X_test, y_test, y_train_augmented, X_train_augmented
del dataset3D_pred
dataset3D_pred_resized = dataset3D_pred_resized.astype('int8')
del dataset3D
gc.collect()

After making the predictions, the data cube can be visualised with 3D meshing using a fast marching algorithm. The 3D results of the segmentation are shown below using an interactive plotly figure. The data has been rebinned by a factor of ~24 to reduce the file size and maintain webpage stability.

In [0]:
from skimage import measure
vertices, faces, normals, values = measure.marching_cubes_lewiner(dataset3D_pred_resized,
                                                                  step_size=24)

In [0]:
enable_plotly_in_cell()

import plotly.figure_factory as ff
x,y,z = zip(*vertices)  
fig = ff.create_trisurf(x=x,y=y,z=z,simplices=faces)
fig['layout'] = dict(
         title="Mitochondria ISO-surface binned by a factor of 12", 
         font=dict(family='Balto'),
         showlegend=False,
         width=900,
         height=900,
         scene=dict(aspectratio=dict(x=dataset3D_pred_resized.shape[0]/dataset3D_pred_resized.shape[2],
                                     y=dataset3D_pred_resized.shape[1]/dataset3D_pred_resized.shape[2], 
                                     z=dataset3D_pred_resized.shape[2]/dataset3D_pred_resized.shape[2])
                    )
        )
iplot(fig)

### Measuring properties from the datacube. 

After model segmentation and visualisation, quantitative size and shape information can be obtained from the sample.
The following measurements can be performed straight forwardly: 
1. Volume
2. Closest nearest neighbours
3. Sphericity (Circularity)
4. Eccentricity
5. Diameter (or radius)

A future post will explore how to measure these properties for tomography datasets.